In [1]:
using Random, Distributions

In [7]:
struct Environment
    hours::Float64
    arrival_D_st1::Exponential
    arrival_D_st2::Exponential
    travel_time_st1st2_D::Exponential
    travel_time_st2st1_D::Exponential

    
    Environment() = new(420.0, 
    Exponential(10),
    Exponential(10),
    Exponential(10),
    Exponential(10))
end

mutable struct State
    station1_vehs::Int32
    station2_vehs::Int32
end

# mutable struct Vect
#     time_st1::Float64
#     time_st2::Float64
#     travel_from_st1::Float64
#     travel_from_st2::Float64

#     Vect() = new(Vector(),Vector(),Vector())
# end

function moving(env::Environment, state::State)
    time_st1 = Vector{Float64}()
    time_st2 = Vector{Float64}()
    time_from_st1 = Vector{Float64}()
    time_from_st2 = Vector{Float64}()

    append!(time_st1,rand(arrival_D_st1))
    append!(time_st2,rand(arrival_D_st2))

    max(time_from_st1,time_from_st2)

end


moving (generic function with 1 method)

In [110]:
time_st1 = Vector{Float64}()
time_st2 = Vector{Float64}()
time_from_st1 = Vector{Float64}()
time_from_st2 = Vector{Float64}()
n_st1_vect = Vector{Int64}()
n_st2_vect = Vector{Int64}()

draw_st1 = rand(Exponential(10))
draw_st2 = rand(Exponential(10))



n_st1 = 5
n_st2 = 5
n_total = 10
n_travel = 0
travel_f_1 = 100
travel_f_2 = 100
shit_1 = 0
shit_2 = 0

push!(time_st1,draw_st1)
push!(time_st2,draw_st2)
push!(time_from_st1,travel_f_1)
push!(time_from_st2,travel_f_2)
push!(n_st1_vect,n_st1)
push!(n_st2_vect,n_st2)


# antager at hvis der ikke er en bil løber folk væk igen og der gåe en ny tid før studerende ankommer

## Absolute garbage!


for i in 1:10
    # Crude as shit!
    if time_st1[1] < time_st2[1] && time_st1[1] < time_from_st1[1] && time_st1[1] < time_from_st2[1] # Mangler at folk står i kø
        println("lon")
        if n_st1 >= 1 && n_st1 < n_travel+n_st1+n_st2 
            n_st1 -= 1
            pushfirst!(n_st1_vect,n_st1)
            travel_f_1 = rand(Exponential(10))
            n_travel +=1
            pushfirst!(time_from_st1,travel_f_1+time_st1[1])
        elseif n_st1 == n_travel+n_st1+n_st2
            
        else
            shit_1 += 1
        end

        draw_st1 = rand(Exponential(10))
        pushfirst!(time_st1,time_st1[1]+draw_st1)

    elseif time_st2[1] < time_st1[1] && time_st2[1] < time_from_st1[1] && time_st2[1] < time_from_st2[1]
        println("aj")
        if n_st2 >= 1 && n_st2 < n_travel+n_st1+n_st2
            n_st2 -= 1
            pushfirst!(n_st2_vect,n_st2)
            travel_f_2 = rand(Exponential(10))
            n_travel += 1
            pushfirst!(time_from_st2,travel_f_2+time_st2[1])
            
        elseif n_st2 == n_travel+n_st1+n_st2
            
        else
            shit_2 += 1
        end

        draw_st2 = rand(Exponential(10))
        pushfirst!(time_st2,time_st2[1]+draw_st2)

    elseif time_st1[1] > time_from_st1[1] && time_st2[1] > time_from_st1[1] && time_from_st1[1] < time_from_st2[1] && n_travel+n_st1+n_st2 == n_total
        n_st2 += 1
        pushfirst!(n_st2_vect,n_st2)
        n_travel -= 1
        println("la")
        pushfirst!(time_from_st1,100)

    elseif time_st1[1] > time_from_st2[1] && time_st2[1] > time_from_st2[1] && time_from_st1[1] > time_from_st2[1] && n_travel+n_st1+n_st2 == n_total
        n_st1 += 1
        pushfirst!(n_st1_vect,n_st1)
        n_travel -= 1
        println("na")
        pushfirst!(time_from_st2,100)
    else
        println("huh")
    end
end


println(time_st1,time_from_st1)


aj
na
lon
la
aj
lon
aj
aj
la
lon
[71.62357998333988, 63.093335020066434, 33.313651003967, 9.832390594385078][64.9515268496291, 100.0, 37.834199381928, 100.0, 18.21233596117552, 100.0]


In [111]:
println(n_st1_vect,n_st2_vect)

[3, 4, 5, 6, 5][3, 2, 3, 4, 5, 4, 5]
